<a href="https://colab.research.google.com/github/chardiwall/DPSNN/blob/main/DPSNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Discription
**Title:** DPSNN (A Differentially Private Spiking Neural Network with Temporal Enhanced Polling).

**Objectives:** Implementation of the [paper](https://arxiv.org/pdf/2205.12718.pdf).

**Discription:** The project is set in the context of the growing field of social robotics, which involves the deployment of robots in human-centric environments. These environments can range from healthcare facilities and educational institutions to customer service and domestic settings. The unique aspect of this project lies in its focus on privacy-preserving mechanisms, an increasingly critical concern in today's data-driven world.

---

SNN framwork: [BrainCog](https://github.com/BrainCog-X/Brain-Cog)

DP framwork: [Opacus](https://github.com/pytorch/opacus)

---
Datasets
- Neuromorphic DataSets:
  - [N-MNIST](https://www.garrickorchard.com/datasets/n-mnist#h.p_ID_38)
  - [CIFAR10-DVS](https://figshare.com/articles/dataset/CIFAR10-DVS_New/4724671/2)

- Static Datasets:
  - [MNIST](https://yann.lecun.com/exdb/mnist/)
  - [CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html)


# Loading Essential Libraries

In [ ]:
!pip install -q braincog
# additional dependencies for working with neuromorphic data.
!pip install git+https://github.com/BrainCog-X/tonic_braincog.git

!pip install -q opacus

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.0/123.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 4.7 MB/s eta 0:00:00


In [14]:
# Mounting Google Drive as drive
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [35]:
import os
import sys
import tqdm
from glob import glob
from zipfile import ZipFile

import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns

import torch
from torch import nn
from torch.utils.data import dataset, dataloader

from torchvision import datasets

from braincog.base.node import BaseNode, LIFNode, IzhNode
from braincog.base.strategy.surrogate import *
from braincog.base.utils.visualization import spike_rate_vis, spike_rate_vis_1d

# Unzipping and Loading Data

## Neuromorphic Data

In [ ]:
# N-MNIST

# unzipping the data
base_dir_path = '/content/drive/MyDrive/Colab Notebooks/DPSNN/N-MNIST/'
train_zip_name = 'Copy of Train'
test_zip_name = 'Copy of Test'

for zip_file in [train_zip_name, test_zip_name]:
  zip_path = os.path.join(base_dir_path, zip_file+'.zip')

  with ZipFile(zip_path, 'r') as zip_ref:
    # extraction_path = os.path.join(base_dir_path, zip_file)
    # if not (os.path.exists(extraction_path)):
    #   os.mkdir(extraction_path)
    zip_ref.extractall(base_dir_path)


In [36]:
# N-MNIST LOADING

In [16]:
# CIFAR10-DVS
base_dir_path = '/content/drive/MyDrive/Colab Notebooks/DPSNN/CIFAR10-DVS'
file_name = 'CIFAR10-DVS.zip'

with ZipFile(os.path.join(base_dir_path, file_name), 'r') as zip_ref:
  zip_ref.extractall(base_dir_path)

## Static Data

In [24]:
# MNIST
mnist_train = datasets.MNIST(root = '.', train=True, transform=None, download=True)
mnist_test = datasets.MNIST(root = '.', train=False, transform=None, download=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 92440592.78it/s]


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 69101936.01it/s]


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 31816511.15it/s]


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 13784753.09it/s]


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [25]:
# CIFAR10
cifar10_train = datasets.CIFAR10('.', train = True, transform = None, download = True)
cifar10_test = datasets.CIFAR10('.', train = False, transform = None, download = True)

100%|██████████| 170498071/170498071 [00:03<00:00, 54271524.85it/s]


Extracting ./cifar-10-python.tar.gz to .
Files already downloaded and verified


# df

In [32]:
import braincog.base.node

In [34]:
braincog.base.node?